In [42]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [43]:
 import torch
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Device: {}\n".format(device))

Device: cpu



In [44]:
import pandas as pd
import pickle
import time
from torch.distributions import Categorical
from torch.distributions.uniform import Uniform
import torch.nn.functional as F
import numpy as np 

from src.synthetic_helpers import *
from src.SWkernels import k_sw_rf
from src.MMDkernels import k_MMD
from src.training import train_multiple_kernels
from src.classifiers import KRR

In [69]:
# GMM setup
nl = [50,250] # samples/GMM 
T = 100 # number of datasets [100,500] 
K = 2 # maximum number of components [2,10]
d = 2 # dimension [2,10]
num_exp = len(nl)
epoch = 5
T_val = 50
T_test = 100

In [70]:
v = False #verbose
dic_res = {"n":nl,"T":T,"K":K,"d":d,"SW2":[], "SW1":[],"MMD":[]}
means, covs, y = sample_moments(T, d, K_max=K)
means_val, covs_val, y_val = sample_moments(T_val, d, K_max=K)
means_test, covs_test, y_test = sample_moments(T_test, d, K_max=K)
for idx_exp in range(num_exp):
    n = nl[idx_exp]
    print("Starting with T= ", T, " n=" , n, " K= ", K, " d= ", d)
    X = sample_from_moments(means, covs, n, K, epoch)
    X_val = sample_from_moments(means_val, covs_val, n, K, epoch)
    X_test = sample_from_moments(means_test, covs_test, n, K, epoch)
    
    if "SW2" in dic_res:
        print("Starting SW2 for {} epochs".format(epoch))
        rmse = sw_training(X, y, X_val, y_val, X_test, y_test, 2, epoch, v) # SW2
        dic_res["SW2"].append(rmse)

    if "SW1" in dic_res:
        print("Starting SW1 for {} epochs".format(epoch))
        rmse = sw_training(X, y, X_val, y_val, X_test, y_test, 1, epoch, v) # SW1
        dic_res["SW1"].append(rmse)
    if "MMD" in dic_res:
        print("Starting MMD for {} epochs".format(epoch))
        rmse = mmd_training(X, y, X_val, y_val, X_test, y_test, epoch, v) # MMD
        dic_res["MMD"].append(rmse)
    pickle.dump(dic_res, open("results/gmm_res.p", "wb" ))

Starting with T=  100  n= 50  K=  2  d=  2
Starting SW2 for 5 epochs
Starting SW1 for 5 epochs
Starting MMD for 5 epochs
Starting with T=  100  n= 250  K=  2  d=  2
Starting SW2 for 5 epochs
Starting SW1 for 5 epochs
Starting MMD for 5 epochs


In [71]:
columns = ["MMD", "SW2", "SW1"]
index = dic_res["n"]
res = np.array([np.array(dic_res[k]) for k in columns])
res_mean = res.mean(axis=-1)
res_std = res.std(axis=-1)
pd.DataFrame(res_mean.T, columns=columns, index=index) 

,MMD,SW2,SW1
50,0.67975,0.494770,0.484796
250,0.67590,0.501903,0.505643


In [72]:
pd.DataFrame(res_std.T, columns=columns, index=index) 

,MMD,SW2,SW1
50,0.077073,0.034674,0.031191
250,0.084611,0.009682,0.018036
